In [1]:
%cd ~/work/deeplearning/

/home/asaeed9/work/deeplearning


In [2]:
from theano.sandbox import cuda
cuda.use('gpu1')
%matplotlib inline
from __future__ import print_function, division
#path="../data/2cat/sample"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
from keras.preprocessing import image, sequence
import os, sys, cv2
from shutil import copyfile, move
from random import shuffle

####
from keras.layers.convolutional import *
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
import datetime

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using Theano backend.


In [3]:
data_dir="/home/asaeed9/work/data/2cat"
path="/home/asaeed9/work/data/2cat/sample/"
results_path = "/home/asaeed9/work/data/2cat/sample/results"
test_path = path + '/test/' #We use all the test data
global last_file_timestamp
%cd ../data/2cat/train

/home/asaeed9/work/data/2cat/train


In [4]:
def adjust_prev_data_sample():
    %mv $path/train/cats/* $data_dir/train/
    %mv $path/train/dogs/* $data_dir/train/
    %mv $path/valid/cats/* $data_dir/train/
    %mv $path/valid/dogs/* $data_dir/train/

#clean previous data
def adjust_prev_data():
    %mv $data_dir/valid/* $data_dir/train/    
    adjust_prev_data_sample()

#move training images
def move_from_test(new_trainset, path, train, validation):
    print(new_trainset[:10])
    
def move_to_test(images):
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(images): os.rename(shuf[i], '../sample/test/' + shuf[i]) 
    %mv ../sample/test/cat*.jpg ../sample/test/cats/
    %mv ../sample/test/dog*.jpg ../sample/test/dogs/      

    
def handle_null(train, validation):
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(train): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd $data_dir/train
    
#copy training images
def copy_samples(train, validation):
    #print("Copying new samples for training...")
    #clean previous data
    adjust_prev_data()
    
    #build validation set
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): os.rename(shuf[i], '../valid/' + shuf[i]) 
    
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(train): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd $data_dir/train

def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

def pred_batch(imgs, classes):
    preds = model.predict(imgs)
    idxs = np.argmax(preds, axis=1)

    print('Shape: {}'.format(preds.shape))
    print('First 5 classes: {}'.format(classes[:5]))
    print('First 5 probabilities: {}\n'.format(preds[:5]))
    print('Predictions prob/class: ')
    
    for i in range(len(idxs)):
        idx = idxs[i]
        print ('  {:.4f}/{}'.format(preds[i, idx], classes[idx]))


def generate_size_graph(fig_no, training_size, accuracy, loss, start_size, end_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(training_size,accuracy)
    plt.plot(training_size,loss)
    plt.xlabel('Training Size')
    plt.ylabel('Accuracy/Loss')
    plt.title('Training Size vs Accuracy/Loss')
    plt.grid(True)
    plt.legend(['Accuracy','Loss'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(path + '/batch_graphs/' +  str(start_size) + '_' + str(end_size) + '.jpg') 
        
def generate_graph(fig_no, epochs, train, val, label, train_title, val_title, train_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(epochs,train)
    plt.plot(epochs,val)
    plt.xlabel('num of Epochs')
    plt.ylabel(label)
    plt.title(train_title + ' vs ' + val_title + '( Samples:' + str(train_size) + ')')
    plt.grid(True)
    plt.legend(['train','val'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(results_path + '/batch_graphs/' +  label + '_' + str(train_size) + '.jpg') 
    
def get_train_model(tr_batches, val_batches, epoch):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model 

def train_model(model, tr_batches, val_batches, epoch):
    if not model:
        model = Sequential([
                BatchNormalization(axis=1, input_shape=(3, 256,256)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
    #             Convolution2D(64,3,3, activation='relu'),
    #             BatchNormalization(axis=1),
                #MaxPooling2D((3,3)),
                Convolution2D(64,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dropout(0.2),
                Dense(2, activation='softmax')
            ])
        model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
       
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
        
    return model 
    
    
def get_test_model():
    model = Sequential([
                BatchNormalization(axis=1, input_shape=(3,256,256)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
    #             Convolution2D(64,3,3, activation='relu'),
    #             BatchNormalization(axis=1),
                #MaxPooling2D((3,3)),
                Convolution2D(64,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dense(2, activation='softmax')
            ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])  
    
    return model

def fit(samples_copied, old_model, path, results_path, nepoch, batch_size, train_size, valid_size):
    gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

    #for train in training_range:
    model = None
    if not samples_copied:
        copy_samples(train_size, valid_size)

    tr_batches = gen_t.flow_from_directory(path + 'train', batch_size=batch_size)
    val_batches = gen_t.flow_from_directory(path + 'valid', class_mode='categorical', shuffle=True, batch_size=batch_size * 2)
    
    if old_model:
        model = train_model(old_model, tr_batches, val_batches, nepoch)
    else:
        model = train_model(None, tr_batches, val_batches, nepoch)
        
    #model.save_weights(results_path+ '/' + 'ft_' + str(train_size) + '.e' + str(nepoch))
    last_file_timestamp = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
    #print('File Time Stamp:{}'.format(last_file_timestamp))
    model.save_weights(results_path+'/ft_{}'.format(last_file_timestamp))
    #model.load_weights(results_path+'/ft_{}'.format(last_file_timestamp))
                          
    return model, last_file_timestamp

def predict(path, model):
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    probs = model.predict(test_data)
    
    #loss_score.append(score[0])
    #accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))
    #print("\nProbabilities:{}".format(probs))
    return probs, test_batches, score[0], score[1]

def move_samples(retrain_set,dest_path, n, limit):
    cats_copied = 0
    dogs_copied = 0
    retrain_list = list(retrain_set)
    shuffle(retrain_list)
    for fil in range(n):
        fil = retrain_list.pop()
        fil_cpy = fil[fil.find('/')+1:]

        if 'cat' in fil_cpy and cats_copied <= limit:
            os.rename(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + dest_path + "/cats/"+ fil_cpy))
            cats_copied+=1
        elif 'dog' in fil_cpy and dogs_copied <= limit:
            os.rename(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + dest_path + "/dogs/" + fil_cpy))
            dogs_copied+=1
            
    #print("Limit:", limit)        
    #print("moved cats:", cats_copied)
    #print("moved dogs:", dogs_copied)
    #print("Retrain Length: ", len(retrain_list))
    return retrain_list, cats_copied, dogs_copied


def move_to_train(retrain_set, limit):
    cats = 0
    dogs = 0
    valid_limit = int(math.floor(.2*(limit*2)))
    train_limit = int(math.floor(.8*(limit*2)))
    
    #print(retrain_set[:10])
    print("validation set: ", valid_limit)
    retrain_list, cats_copied, dogs_copied = move_samples(retrain_set, "valid", valid_limit, limit)
    valid_size = cats_copied + dogs_copied
    print("Train set: ", train_limit)
    retrain_list, cats_copied, dogs_copied = move_samples(retrain_list, "train", train_limit, limit)
    train_size = cats_copied + dogs_copied
    
    return train_size, valid_size, train_size + valid_size

def refil_test(nimages):
    os.chdir("/home/asaeed9/work/data/2cat/train")
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(nimages): os.rename(shuf[i], '../sample/test/' + shuf[i])
    #os.chdir("../sample/test/")
    #move("cat*.jpg")
    %mv ../sample/test/cat*.jpg ../sample/test/cats/
    %mv ../sample/test/dog*.jpg ../sample/test/dogs/     

In [5]:
%cd $data_dir/train

nepoch = 30
batch_size = 64
train_size = 100
retrain_sample_size = 100
running_train_size = 100
training_set_size = []
valid_size = int(math.floor(.2 * train_size))
#print('sample size: {}'.format(train_size + valid_size))
tr_model = None
loss = 0.0
loss_array = []
accuracy = 0.0
accuracy_array = []

#copy test data
#move_to_test(3000)

i=0
#for i in range(100):
print("Train Size:{}".format(train_size))
print("Valid Size:{}".format(valid_size))

if train_size == 0 or valid_size == 0: #handle null case
    handle_null(50,10)
    train_size += 50
    valid_size += 10

tr_model,file_timestamp = fit(i, tr_model, path, results_path, nepoch, batch_size, train_size, valid_size)

model = None
model = get_test_model() 
#model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(nepoch))
#print('{0}/ft_{1}'.format(last_file_timestamp))
#print('Last File Timestamp- before loading:{}'.format(file_timestamp))
model.load_weights(results_path+'/ft_{}'.format(file_timestamp))

probs, test_batches,loss, accuracy = predict(path, model)
training_set_size.append(running_train_size)
loss_array.append(loss)
accuracy_array.append(accuracy)


#get the top 100, most confused images
retrain_idx = np.argsort(probs[:,0] - probs[:, 1])[:retrain_sample_size]

retrain_set = [test_batches.filenames[i] for i in retrain_idx]

os.chdir(path + 'test')
ndog = sum('dog' in name for name in retrain_set)
ncat =  sum('cat' in name for name in retrain_set)
limit = min(ncat, ndog)

#move existing training data to the store
adjust_prev_data_sample()
train_size, valid_size, copied_images = move_to_train(retrain_set, limit)
running_train_size += train_size
refil_test(copied_images)

print('Training Set Size:{}'.format(training_set_size))
print('Accuracy:{}'.format(accuracy_array))
print('Loss:{}'.format(loss_array))

/home/asaeed9/work/data/2cat/train
Train Size:100
Valid Size:20
/home/asaeed9/work/data/2cat/valid
/home/asaeed9/work/data/2cat/train
Found 100 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/2
100/100 [==============================] - 1s - loss: 0.8525 - acc: 0.5500 - val_loss: 1.4459 - val_acc: 0.7000
Epoch 2/2
100/100 [==============================] - 1s - loss: 0.9482 - acc: 0.5100 - val_loss: 0.9429 - val_acc: 0.6000
Epoch 1/28
100/100 [==============================] - 1s - loss: 0.9513 - acc: 0.6300 - val_loss: 0.7411 - val_acc: 0.6000
Epoch 2/28
100/100 [==============================] - 1s - loss: 0.9643 - acc: 0.5500 - val_loss: 0.7594 - val_acc: 0.5000
Epoch 3/28
100/100 [==============================] - 1s - loss: 1.0104 - acc: 0.5600 - val_loss: 0.7829 - val_acc: 0.4500
Epoch 4/28
100/100 [==============================] - 1s - loss: 0.8822 - acc: 0.6400 - val_loss: 0.8724 - val_acc: 0.3000
Epoch 5/28
100/100 [=============================

In [6]:
sample_probs = probs[:3, :]

np.argsort(sample_probs)

array([[1, 0],
       [1, 0],
       [1, 0]])

In [7]:
sample_probs

array([[ 0.8215,  0.1785],
       [ 0.7075,  0.2925],
       [ 0.6634,  0.3366]], dtype=float32)

In [8]:
sample_probs[:3,0] - sample_probs[:3, 1]

array([ 0.6431,  0.4149,  0.3267], dtype=float32)

In [10]:
np.argsort(sample_probs[:3,0] - sample_probs[:3, 1])[:2]

array([2, 1])

In [ ]:
sorted(probs[:2,0] - probs[:2,1])

In [ ]:

np.where(sorted(abs(probs[:2,0] - probs[:2,1])))[0]

#np.where(np.logical_and(probs >=0.4, probs<=0.7))[0]
#np.where()[10]

In [ ]:
np.where(abs(probs[:,0] - probs[:, 1]))[0]

In [ ]:
Training Set Size:[100, 255, 507, 544, 586, 669, 727, 786, 829, 1097, 1166, 1288, 1322, 1610, 1795, 1979, 2124, 2384, 2555, 2707, 2903, 3015, 3078, 3151, 3306, 3424, 3480, 3544, 3614, 3762, 3984, 4203, 4355, 4502, 4638, 4758, 4830, 4918, 4974, 5002, 5014, 5049, 5083, 5108, 5121, 5163, 5196, 5271, 5297, 5338, 5377, 5442, 5478, 5585, 5647, 5744, 5804, 5842, 5885, 5926, 5964, 5990, 5996, 6050, 6098, 6114, 6137, 6156, 6292, 6360, 6493, 6603, 6641, 6672, 6705, 6740, 6856, 6885, 6969, 7022, 7057, 7103, 7119, 7137, 7157, 7167, 7170, 7192, 7259, 7279, 7326, 7368, 7409, 7489, 7508, 7519, 7535, 7565, 7577, 7593]
Accuracy:[0.51834556370913942, 0.53235490327878665, 0.54069379586390931, 0.55103402269172863, 0.56370913944616563, 0.56170780524323194, 0.57171447635730799, 0.57738492332195102, 0.59472981993956431, 0.607738492358041, 0.61674449629112427, 0.62408272182448377, 0.65310206809506721, 0.6711140761103449, 0.67645096735130394, 0.69246164113382569, 0.68478985995630803, 0.70180120085023734, 0.71247498336197779, 0.72214809877225128, 0.70713809210113721, 0.71180787194443196, 0.72014676454943605, 0.72481654442255306, 0.71581054040000314, 0.72581721151407919, 0.72314876587371812, 0.72848565714449942, 0.73048699128778871, 0.73982655113342999, 0.74583055378199414, 0.76951300877185569, 0.77451634432889371, 0.77718478995931395, 0.76484322889873868, 0.77818545695143271, 0.78118745826577407, 0.77318212137451325, 0.76751167440987011, 0.77151434285550058, 0.77384923278211837, 0.77985323556985275, 0.77451634418972337, 0.78452301542308822, 0.77585056700493349, 0.77384923278211837, 0.7795196797467614, 0.78685790532982458, 0.77985323545056395, 0.76984656442595456, 0.78519012681081191, 0.78952635096024482, 0.79319546360266535, 0.80453635753195152, 0.80553702464335919, 0.7871914609541012, 0.79719813204829582, 0.80020013338251861, 0.79786524345590082, 0.79619746493688814, 0.78819212804562733, 0.79452968641787547, 0.79853235486350593, 0.79919946639039974, 0.80086724479012361, 0.79886591056730849, 0.79986657767871605, 0.71881254179387033, 0.79686457634449326, 0.77484989991340736, 0.8115410273316862, 0.81387591735771136, 0.81087391602348857, 0.81187458303548876, 0.81120747174717256, 0.80820547029366085, 0.81654436288872423, 0.8042028018480305, 0.81621080728432904, 0.82154769854516962, 0.8232154769448935, 0.82188125412968338, 0.81420947306151392, 0.81420947306151392, 0.79786524357518962, 0.79919946627111094, 0.81854569721094683, 0.81387591735771136, 0.81454302866590911, 0.79953302199479503, 0.81320880596998768, 0.81521014007351411, 0.82088058715744605, 0.81154102731180477, 0.81120747172729102, 0.81420947296210655, 0.81854569711153946, 0.81621080728432904, 0.81721147427644791, 0.81187458301560733]
Loss:[0.94627340202573307, 1.069129374521903, 1.6582751717068021, 1.9458960867508639, 1.7672291246949234, 1.8073495392544099, 2.2642813132276847, 2.1801735833585463, 1.4026525458786947, 1.9107522724806427, 2.0852411977643568, 1.885862276673794, 1.4222142591804088, 1.6198320426966366, 1.4607558314246762, 1.637531633791405, 1.6699402961753225, 1.4755695811345468, 1.4915868397193244, 1.3634724204066277, 1.7743482206655472, 1.8913224160949575, 1.6106715754042313, 1.3743048965970701, 1.565028794591947, 1.9099577603696425, 1.9002086268719274, 1.8303540087018593, 1.4808997903727468, 1.0050827415884933, 1.0144933128929521, 1.0415886528218405, 1.0145420589750811, 1.1418929995895626, 1.1928224458226846, 1.4710475567024974, 1.341952260054295, 1.8395282249944938, 2.3651828057511031, 2.3124792167279753, 2.305378243448915, 2.0921286372747461, 2.2739992053878604, 2.3005914534378444, 2.2752316740566392, 2.1230344122071925, 1.8697313540941323, 1.9767550084891519, 2.3808871660582858, 2.0304581639465638, 1.6816643132226141, 1.7091403857570606, 1.255017541860882, 1.4477932849790829, 1.3391912613831973, 1.695395214437166, 1.6405188316495478, 1.883696767807832, 1.8168773716272137, 2.1832812890025841, 2.5262706354183559, 2.199554436640943, 1.8084424575585736, 1.9852745032712713, 1.9270797232017987, 1.9817351747420384, 2.3064219975595739, 1.7742115364939948, 1.6118588206358875, 1.3782198212002021, 1.2986096903791102, 1.6194246127093093, 1.8720587353657601, 1.7112238509897353, 1.764265245251641, 1.3266987558938885, 1.6663410443420572, 1.6504295845829779, 1.7126746936698132, 1.9551829461497892, 1.7929846915162309, 2.0409724012221035, 2.1942079734076447, 2.3111469137771712, 2.2224564307727781, 2.5554216508963821, 2.0883331067482378, 1.6952288752798241, 2.1873511853298768, 2.0337138844416631, 1.9773875592814496, 1.9745328070035451, 1.6287168510223404, 2.3116104998768714, 2.2300192884155075, 2.162166691486481, 2.0174554670189169, 2.2769914772579942, 2.1841097289998905, 2.3013370820185077]

In [ ]:
generate_size_graph(1, training_set_size, accuracy_array, loss_array, min(training_set_size), max(training_set_size))